In [54]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.formula.api as smf
import scipy.stats as stats
import pandas_profiling   #need to install using anaconda prompt (pip install pandas_profiling)

%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 7.5
plt.rcParams['axes.grid'] = True
plt.gray()

from matplotlib.backends.backend_pdf import PdfPages

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor

<Figure size 720x540 with 0 Axes>

In [55]:
# Load the data
bankloans = pd.read_csv( 'bankloans.csv' )

bankloans.head()

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default
0,41,3,17,12,176,9.3,11.359392,5.008608,1.0
1,27,1,10,6,31,17.3,1.362202,4.000798,0.0
2,40,1,15,14,55,5.5,0.856075,2.168925,0.0
3,41,1,15,14,120,2.9,2.658720,0.821280,0.0
4,24,2,2,0,28,17.3,1.787436,3.056564,1.0


In [4]:
pr = pandas_profiling.ProfileReport(bankloans)
pr.to_file('Practice-profileReport.html')

Summarize dataset:   0%|          | 0/22 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [56]:
bankloans_existing = bankloans[bankloans.default.isnull()==0] #It gives the details of existing customers
bankloans_new = bankloans[bankloans.default.isnull()==1] #It gives the details of new customers

In [57]:
print(bankloans_existing)
print()
print(bankloans_new)

     age  ed  employ  address  income  debtinc   creddebt   othdebt  default
0     41   3      17       12     176      9.3  11.359392  5.008608      1.0
1     27   1      10        6      31     17.3   1.362202  4.000798      0.0
2     40   1      15       14      55      5.5   0.856075  2.168925      0.0
3     41   1      15       14     120      2.9   2.658720  0.821280      0.0
4     24   2       2        0      28     17.3   1.787436  3.056564      1.0
..   ...  ..     ...      ...     ...      ...        ...       ...      ...
695   36   2       6       15      27      4.6   0.262062  0.979938      1.0
696   29   2       6        4      21     11.5   0.369495  2.045505      0.0
697   33   1      15        3      32      7.6   0.491264  1.940736      0.0
698   45   1      19       22      77      8.4   2.302608  4.165392      0.0
699   37   1      12       14      44     14.7   2.994684  3.473316      0.0

[700 rows x 9 columns]

     age  ed  employ  address  income  debtinc  cre

In [7]:
#Missing value treatment
def missing_treat(x):
    x = x.fillna(x.median())
    return x

bankloans_existing = bankloans_existing.apply(lambda x: missing_treat(x))
bankloans_existing

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default
0,41,3,17,12,176,9.3,11.359392,5.008608,1.0
1,27,1,10,6,31,17.3,1.362202,4.000798,0.0
2,40,1,15,14,55,5.5,0.856075,2.168925,0.0
3,41,1,15,14,120,2.9,2.658720,0.821280,0.0
4,24,2,2,0,28,17.3,1.787436,3.056564,1.0
...,...,...,...,...,...,...,...,...,...
695,36,2,6,15,27,4.6,0.262062,0.979938,1.0
696,29,2,6,4,21,11.5,0.369495,2.045505,0.0
697,33,1,15,3,32,7.6,0.491264,1.940736,0.0
698,45,1,19,22,77,8.4,2.302608,4.165392,0.0


In [8]:
#Handling Outliers - Method2
def outlier_capping(x):
    x = x.clip(upper = x.quantile(0.99))
    x = x.clip(lower = x.quantile(0.01))
    return x

bankloans_existing = bankloans_existing.apply(lambda x: outlier_capping(x))
bankloans_existing

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default
0,41.0,3,17.0,12.0,176.0,9.3,9.891428,5.008608,1.0
1,27.0,1,10.0,6.0,31.0,17.3,1.362202,4.000798,0.0
2,40.0,1,15.0,14.0,55.0,5.5,0.856075,2.168925,0.0
3,41.0,1,15.0,14.0,120.0,2.9,2.658720,0.821280,0.0
4,24.0,2,2.0,0.0,28.0,17.3,1.787436,3.056564,1.0
...,...,...,...,...,...,...,...,...,...
695,36.0,2,6.0,15.0,27.0,4.6,0.262062,0.979938,1.0
696,29.0,2,6.0,4.0,21.0,11.5,0.369495,2.045505,0.0
697,33.0,1,15.0,3.0,32.0,7.6,0.491264,1.940736,0.0
698,45.0,1,19.0,22.0,77.0,8.4,2.302608,4.165392,0.0


In [ ]:
# 	Information value			WOE		IV = (%1's - %0's)*WOE		
# Age Bins	#0	#1	Total	log(odds)	%1's	%0's		IV
# <20	100	50	150	-0.301029996	0.25	0.125		-0.037628749
# 20-35	50	50	100	0	0.25	0.0625		0
# 35-50	20	30	50	0.176091259	0.15	0.025		0.022011407
# 50-60	280	20	300	-1.146128036	0.1	0.35		0.286532009
# 60+	350	50	400	-0.84509804	0.25	0.4375		0.158455883
# 								0.429370549
# 				IV				
# 				<0.2	Not good			
# 				0.2-03	week relationship			
# 				0.3-0.5	strong relationship			
# 				0.5+	very strong or fishy			


In [9]:
#Information value calculation
def calculate_woe_iv(dataset, feature, target):
    lst = []
    for i in range(dataset[feature].nunique()):
        val = list(dataset[feature].unique())[i]
        lst.append({
            'Value': val,
            'All': dataset[dataset[feature] == val].count()[feature],
            'Good': dataset[(dataset[feature] == val) & (dataset[target] == 0)].count()[feature],
            'Bad': dataset[(dataset[feature] == val) & (dataset[target] == 1)].count()[feature]
        })
        
    dset = pd.DataFrame(lst)
    dset['Distr_Good'] = dset['Good'] / dset['Good'].sum()
    dset['Distr_Bad'] = dset['Bad'] / dset['Bad'].sum()
    dset['WoE'] = np.log(dset['Distr_Good'] / dset['Distr_Bad'])
    dset = dset.replace({'WoE': {np.inf: 0, -np.inf: 0}})
    dset['IV'] = (dset['Distr_Good'] - dset['Distr_Bad']) * dset['WoE']
    iv = dset['IV'].sum()
    
    dset = dset.sort_values(by='WoE')
    
    return dset, iv

In [10]:
for col in bankloans_existing.columns:
    if col == 'default': continue
    else:
        print('WoE and IV for column: {}'.format(col))
        df, iv = calculate_woe_iv(bankloans_existing, col, 'default')
        print(df)
        print('IV score: {:.2f}'.format(iv))
        print('\n')

WoE and IV for column: age
    Value  All  Good  Bad  Distr_Good  Distr_Bad       WoE        IV
28  53.00    6     2    4    0.003868   0.021858 -1.731704  0.031152
20  23.00   18     8   10    0.015474   0.054645 -1.261700  0.049422
3   24.00   24    12   12    0.023211   0.065574 -1.038557  0.043996
7   25.00   20    11    9    0.021277   0.049180 -0.837886  0.023380
29  53.01    7     4    3    0.007737   0.016393 -0.750875  0.006500
30  22.00   12     7    5    0.013540   0.027322 -0.702084  0.009677
12  28.00   37    23   14    0.044487   0.076503 -0.542120  0.017356
19  32.00   25    16    9    0.030948   0.049180 -0.463193  0.008445
1   27.00   28    18   10    0.034816   0.054645 -0.450770  0.008938
14  21.00   12     8    4    0.015474   0.021858 -0.345410  0.002205
9   37.00   22    15    7    0.029014   0.038251 -0.276417  0.002553
13  29.00   44    30   14    0.058027   0.076503 -0.276417  0.005107
11  47.00   16    11    5    0.021277   0.027322 -0.250099  0.001512
8   52.

C:\Users\vishalk\Anaconda3\lib\site-packages\pandas\core\arraylike.py:274: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


    Value  All  Good  Bad  Distr_Good  Distr_Bad       WoE        IV
71   70.0    4     1    3    0.001934   0.016393 -2.137169  0.030902
32   50.0    6     2    4    0.003868   0.021858 -1.731704  0.031152
35   15.0   14     5    9    0.009671   0.049180 -1.626343  0.064255
60  102.0    2     1    1    0.001934   0.005464 -1.038557  0.003666
96   86.0    2     1    1    0.001934   0.005464 -1.038557  0.003666
..    ...  ...   ...  ...         ...        ...       ...       ...
45   39.0   14    12    2    0.023211   0.010929  0.753203  0.009251
62   56.0    7     6    1    0.011605   0.005464  0.753203  0.004625
59   30.0   16    14    2    0.027079   0.010929  0.907353  0.014654
22   32.0   17    15    2    0.029014   0.010929  0.976346  0.017657
42   44.0   12    11    1    0.021277   0.005464  1.359339  0.021494

[107 rows x 8 columns]
IV score: 0.40


WoE and IV for column: debtinc


C:\Users\vishalk\Anaconda3\lib\site-packages\pandas\core\arraylike.py:274: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


      Value  All  Good  Bad  Distr_Good  Distr_Bad       WoE        IV
95   30.701    7     1    6    0.001934   0.032787 -2.830316  0.087323
62   23.100    5     1    4    0.001934   0.021858 -2.424851  0.048312
32   16.100    5     1    4    0.001934   0.021858 -2.424851  0.048312
138  19.900    4     1    3    0.001934   0.016393 -2.137169  0.030902
117  12.500    3     1    2    0.001934   0.010929 -1.731704  0.015576
..      ...  ...   ...  ...         ...        ...       ...       ...
51    5.000    7     6    1    0.011605   0.005464  0.753203  0.004625
66    0.900    8     7    1    0.013540   0.005464  0.907353  0.007327
48    3.700    8     7    1    0.013540   0.005464  0.907353  0.007327
71    5.400    9     8    1    0.015474   0.005464  1.040885  0.010419
23    6.700    9     8    1    0.015474   0.005464  1.040885  0.010419

[221 rows x 8 columns]
IV score: 0.59


WoE and IV for column: creddebt


C:\Users\vishalk\Anaconda3\lib\site-packages\pandas\core\arraylike.py:274: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


        Value  All  Good  Bad  Distr_Good  Distr_Bad       WoE        IV
253  0.449820    2     1    1    0.001934   0.005464 -1.038557  0.003666
451  0.276360    1     1    0    0.001934   0.000000  0.000000  0.000000
452  0.488250    1     0    1    0.000000   0.005464  0.000000 -0.000000
453  0.121576    1     1    0    0.001934   0.000000  0.000000  0.000000
454  0.602040    1     1    0    0.001934   0.000000  0.000000  0.000000
..        ...  ...   ...  ...         ...        ...       ...       ...
230  1.606500    1     1    0    0.001934   0.000000  0.000000  0.000000
231  0.563200    1     0    1    0.000000   0.005464  0.000000 -0.000000
232  0.103488    1     1    0    0.001934   0.000000  0.000000  0.000000
224  4.039200    1     0    1    0.000000   0.005464  0.000000 -0.000000
682  2.994684    1     1    0    0.001934   0.000000  0.000000  0.000000

[683 rows x 8 columns]
IV score: 0.00


WoE and IV for column: othdebt
         Value  All  Good  Bad  Distr_Good  Distr_Ba

C:\Users\vishalk\Anaconda3\lib\site-packages\pandas\core\arraylike.py:274: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [11]:
#From above step, we can find important variables: Age (0.41), employ(0.60), address(0.46), income(0.40), debtincome(0.59)

In [58]:
model = smf.logit('default~age', data=bankloans_existing).fit()

Optimization terminated successfully.
         Current function value: 0.564806
         Iterations 5


In [59]:
model.predict(bankloans_existing) #It will give us the probability.

0      0.211433
1      0.322667
2      0.218359
3      0.211433
4      0.350150
         ...   
695    0.247678
696    0.304989
697    0.271333
698    0.185348
699    0.240108
Length: 700, dtype: float64

In [13]:
2*metrics.roc_auc_score(bankloans_existing.default, model.predict(bankloans_existing))-1

0.20027269556394067

In [14]:
gini_df = pd.DataFrame()
for col in bankloans_existing.columns.difference(['default']):
    model = smf.logit('default~'+str(col), data=bankloans_existing).fit()
    gini = 2*metrics.roc_auc_score(bankloans_existing.default, model.predict(bankloans_existing))-1
    temp = pd.DataFrame([col, gini]).T
    gini_df = pd.concat([gini_df, temp], axis=0)

Optimization terminated successfully.
         Current function value: 0.559856
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.564673
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.548956
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.501389
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.567774
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.527166
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.569055
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.565619
         Iterations 5


In [15]:
gini_df.columns= ['Feature', 'SomerceD']
gini_df.sort_values(by='SomerceD', ascending=False, inplace=True)

In [16]:
gini_df.to_csv('Practice-gini_df.csv')

In [17]:
#RFE
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
X = bankloans_existing[bankloans_existing.columns.difference(['default'])]
y = bankloans_existing[['default']]

rfe = RFE(RandomForestClassifier(), 5)
rfe = rfe.fit(X, y)

X.columns[rfe.get_support()]

C:\Users\vishalk\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_features_to_select=5 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
C:\Users\vishalk\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\vishalk\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\vishalk\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array

Index(['creddebt', 'debtinc', 'employ', 'income', 'othdebt'], dtype='object')

In [18]:
#SelectKbest
from sklearn.feature_selection import SelectKBest, f_classif
SKB = SelectKBest(f_classif, k=5).fit(X, y )

X.columns[SKB.get_support()]

C:\Users\vishalk\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Index(['address', 'creddebt', 'debtinc', 'employ', 'othdebt'], dtype='object')

In [19]:
Final_list = [
#'age',
'employ',
'address',
#'income',
'debtinc',
'creddebt',
'othdebt'
]

In [20]:
X_new = X[Final_list]
X_new

,employ,address,debtinc,creddebt,othdebt
0,17.0,12.0,9.3,9.891428,5.008608
1,10.0,6.0,17.3,1.362202,4.000798
2,15.0,14.0,5.5,0.856075,2.168925
3,15.0,14.0,2.9,2.658720,0.821280
4,2.0,0.0,17.3,1.787436,3.056564
...,...,...,...,...,...
695,6.0,15.0,4.6,0.262062,0.979938
696,6.0,4.0,11.5,0.369495,2.045505
697,15.0,3.0,7.6,0.491264,1.940736
698,19.0,22.0,8.4,2.302608,4.165392


In [21]:
### VIF Calculation for variables
vif = pd.DataFrame()
vif["VIF_Factor"] = [variance_inflation_factor(X_new.values, i) for i in range(X_new.shape[1])]
vif["features"] = X_new.columns

vif.sort_values(by='VIF_Factor',ascending=False)

,VIF_Factor,features
4,4.436528,othdebt
2,3.624147,debtinc
3,3.269440,creddebt
0,3.073768,employ
1,2.481654,address


In [22]:
data_final = pd.concat([X_new, y], axis=1)

data_final

,employ,address,debtinc,creddebt,othdebt,default
0,17.0,12.0,9.3,9.891428,5.008608,1.0
1,10.0,6.0,17.3,1.362202,4.000798,0.0
2,15.0,14.0,5.5,0.856075,2.168925,0.0
3,15.0,14.0,2.9,2.658720,0.821280,0.0
4,2.0,0.0,17.3,1.787436,3.056564,1.0
...,...,...,...,...,...,...
695,6.0,15.0,4.6,0.262062,0.979938,1.0
696,6.0,4.0,11.5,0.369495,2.045505,0.0
697,15.0,3.0,7.6,0.491264,1.940736,0.0
698,19.0,22.0,8.4,2.302608,4.165392,0.0


In [23]:
#split the data into train & test
train, test = train_test_split(data_final, test_size=0.3, random_state=123)

In [24]:
train.shape

(490, 6)

In [25]:
test.shape

(210, 6)

In [26]:
'default~'+'+'.join(train.columns.difference(['default']))

'default~address+creddebt+debtinc+employ+othdebt'

In [27]:
formula = 'default~address+creddebt+debtinc+employ'

In [28]:
model = smf.logit(formula, data=train).fit()

Optimization terminated successfully.
         Current function value: 0.400953
         Iterations 7


In [29]:
print(model.summary())

                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                  490
Model:                          Logit   Df Residuals:                      485
Method:                           MLE   Df Model:                            4
Date:                Tue, 16 Mar 2021   Pseudo R-squ.:                  0.3118
Time:                        11:50:56   Log-Likelihood:                -196.47
converged:                       True   LL-Null:                       -285.50
Covariance Type:            nonrobust   LLR p-value:                 1.944e-37
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.8974      0.299     -3.000      0.003      -1.484      -0.311
address       -0.0724      0.024     -2.995      0.003      -0.120      -0.025
creddebt       0.6506      0.106      6.146      0.0

In [30]:
#Metrics based on the probability - AUC, Gini
train['prob'] = model.predict(train)
test['prob'] = model.predict(test)

<ipython-input-30-2e6b4b73df87>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['prob'] = model.predict(train)
<ipython-input-30-2e6b4b73df87>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['prob'] = model.predict(test)


In [31]:
train.head()

,employ,address,debtinc,creddebt,othdebt,default,prob
404,18.0,10.0,15.400,2.099636,3.136364,0.0,0.034050
63,4.0,9.0,7.000,2.159850,1.690150,0.0,0.372982
34,8.0,1.0,2.900,0.077140,0.937860,0.0,0.066860
33,8.0,4.0,14.400,1.018656,2.869344,0.0,0.224398
583,16.0,14.0,30.701,7.320000,10.980000,1.0,0.829851


In [32]:
#Calculating AUC for train & Test
train_auc = metrics.roc_auc_score(train.default, train.prob)
train_auc

0.8562510580667005

In [33]:
test_auc = metrics.roc_auc_score(test.default, test.prob)
test_auc

0.8403009002343076

In [34]:
#Calculating Gini (somerceD) for train & Test
train_gini = 2*metrics.roc_auc_score(train.default, train.prob)-1
train_gini

0.7125021161334011

In [35]:
test_gini = 2*metrics.roc_auc_score(test.default, test.prob)-1
test_gini

0.6806018004686152

In [36]:
temp = train
roc_df = pd.DataFrame()
for cut_off in np.linspace(0,1):
    temp['y_pred'] = np.where(train.prob>cut_off, 1, 0)
    temp['TP'] = np.where(((train.default ==1) & (train.y_pred==1)), 1,0)
    temp['TN'] = np.where(((train.default ==0) & (train.y_pred==0)), 1,0)
    temp['FP'] = np.where(((train.default ==0) & (train.y_pred==1)), 1,0)
    temp['FN'] = np.where(((train.default ==1) & (train.y_pred==0)), 1,0)
    sensitivity = temp.TP.sum()/(temp.TP.sum()+temp.FN.sum())
    specificity = temp.TN.sum()/(temp.TN.sum()+temp.FP.sum())
    accuracy = (temp.TN.sum()+temp.TP.sum())/(temp.y_pred.count())
    temp_df = pd.DataFrame([cut_off, sensitivity, specificity, accuracy]).T
    temp_df.columns = ['cutoff', 'sensitivity', 'specificity', 'accuracy']
    roc_df = pd.concat([roc_df, temp_df], axis=0)

<ipython-input-36-ac65d9c20593>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['y_pred'] = np.where(train.prob>cut_off, 1, 0)
<ipython-input-36-ac65d9c20593>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['TP'] = np.where(((train.default ==1) & (train.y_pred==1)), 1,0)
<ipython-input-36-ac65d9c20593>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

<ipython-input-36-ac65d9c20593>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['FP'] = np.where(((train.default ==0) & (train.y_pred==1)), 1,0)
<ipython-input-36-ac65d9c20593>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['FN'] = np.where(((train.default ==1) & (train.y_pred==0)), 1,0)
<ipython-input-36-ac65d9c20593>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

<ipython-input-36-ac65d9c20593>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['TP'] = np.where(((train.default ==1) & (train.y_pred==1)), 1,0)
<ipython-input-36-ac65d9c20593>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['TN'] = np.where(((train.default ==0) & (train.y_pred==0)), 1,0)
<ipython-input-36-ac65d9c20593>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [37]:
roc_df['total'] = roc_df.sensitivity + roc_df.specificity

In [38]:
roc_df.sort_values(by = 'total', ascending=False).head(1)

,cutoff,sensitivity,specificity,accuracy,total
0,0.265306,0.795455,0.756983,0.767347,1.552438


In [39]:
roc_df[roc_df.total == roc_df.total.max()]

,cutoff,sensitivity,specificity,accuracy,total
0,0.265306,0.795455,0.756983,0.767347,1.552438


In [40]:
#From above step, the cut-off is: 0.265306
train['default_pred'] = np.where(train.prob>0.265306, 1, 0)
test['default_pred'] = np.where(test.prob>0.265306, 1, 0)

<ipython-input-40-6dcf45e40f5c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['default_pred'] = np.where(train.prob>0.265306, 1, 0)
<ipython-input-40-6dcf45e40f5c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['default_pred'] = np.where(test.prob>0.265306, 1, 0)


In [41]:
metrics.confusion_matrix(train.default, train.default_pred)

array([[271,  87],
       [ 27, 105]], dtype=int64)

In [42]:
#Good ness of fit metrics based on categorical predicted output for train & test

print(metrics.classification_report(train.default, train.default_pred))

              precision    recall  f1-score   support

         0.0       0.91      0.76      0.83       358
         1.0       0.55      0.80      0.65       132

    accuracy                           0.77       490
   macro avg       0.73      0.78      0.74       490
weighted avg       0.81      0.77      0.78       490



In [43]:
print(metrics.classification_report(test.default, test.default_pred))

              precision    recall  f1-score   support

         0.0       0.91      0.77      0.83       159
         1.0       0.51      0.76      0.61        51

    accuracy                           0.77       210
   macro avg       0.71      0.77      0.72       210
weighted avg       0.81      0.77      0.78       210



In [44]:
#Decile Analysis
train['Deciles']=pd.qcut(train['prob'],10, labels=False)
test['Deciles']=pd.qcut(test['prob'],10, labels=False)

<ipython-input-44-fc5825defa97>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Deciles']=pd.qcut(train['prob'],10, labels=False)
<ipython-input-44-fc5825defa97>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Deciles']=pd.qcut(test['prob'],10, labels=False)


In [45]:
train['goods'] = 1-train.default
test['goods'] = 1-test.default

<ipython-input-45-dab1d3814fda>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['goods'] = 1-train.default
<ipython-input-45-dab1d3814fda>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['goods'] = 1-test.default


In [46]:
decile_results_train = train.groupby(['Deciles']).agg(min_prob = ('prob', 'min'),
                              max_prob = ('prob', 'max'),
                              No_bads = ('default', 'sum'),
                              No_goods = ('goods', 'sum'), 
                              total = ('default', 'count'))

decile_results_test = test.groupby(['Deciles']).agg(min_prob = ('prob', 'min'),
                              max_prob = ('prob', 'max'),
                              No_bads = ('default', 'sum'),
                              No_goods = ('goods', 'sum'), 
                              total = ('default', 'count'),)

In [47]:
decile_results_train.to_csv('Practice-decile_results_train.csv')
decile_results_test.to_csv('Practice-decile_results_test.csv')

In [48]:

bankloans_new['prob']  = model.predict(bankloans_new)

<ipython-input-48-c07090ab28f4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bankloans_new['prob']  = model.predict(bankloans_new)


In [49]:
bankloans_new['default_predict'] = np.where(bankloans_new.prob>0.265306,1,0)

<ipython-input-49-4d2d6932b7c1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bankloans_new['default_predict'] = np.where(bankloans_new.prob>0.265306,1,0)


In [50]:
bankloans_new.default_predict.value_counts()

0    95
1    55
Name: default_predict, dtype: int64

In [51]:
bankloans_new['creditscore'] = 1000-bankloans_new.prob*1000

<ipython-input-51-731093787fb8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bankloans_new['creditscore'] = 1000-bankloans_new.prob*1000


In [52]:
bankloans_new.head()

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default,prob,default_predict,creditscore
700,36,1,16,13,32,10.9,0.544128,2.943872,NaN,0.011274,0,988.725731
701,50,1,6,27,21,12.9,1.316574,1.392426,NaN,0.087015,0,912.985451
702,40,1,9,9,33,17.0,4.880700,0.729300,NaN,0.708953,1,291.046934
703,31,1,5,7,23,2.0,0.046000,0.414000,NaN,0.080848,0,919.151555
704,29,1,4,0,24,7.8,0.866736,1.005264,NaN,0.345418,1,654.581963
